In [1]:
from __future__ import absolute_import,division,print_function
import sys
sys.path.append('../tfmodels')

import numpy as np
import tensorflow as tf
from user2vec import *
#from tensorflow.contrib.rnn.python.ops import core_rnn_cell as rnn_cell
#from tensorflow.python.ops import rnn
#from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn,dynamic_rnn
#from tensorflow.contrib.rnn.python.ops.core_rnn_cell import RNNCell,LSTMCell,LSTMStateTuple

#from data import read_data
#from memory_model import *
import numpy as np
import tensorflow as tf
from past.builtins import xrange
from sklearn.model_selection import train_test_split
import h5py
import math

In [2]:
x_filename = '/home/host/CORP/wei.wang/data_user2vec_train.h5'

h5f = h5py.File(x_filename, 'r')
show = h5f['show']
isMovie = h5f['isMovie']
time = h5f['time']
nextShow = h5f['nextShow']
nextTime = h5f['nextTime']
isTheSame = h5f['isTheSame']
cancellation = h5f['cancellation']
seq_len = np.zeros(len(show)) + 50


In [3]:
flags = tf.app.flags

flags.DEFINE_integer("nb_words", 29000, "term number in input sequence(zero mask) [20001]")
flags.DEFINE_integer("maxlen", 50, "the max length of input sequence [80]")
flags.DEFINE_integer("num_layers", 1, "the number of rnn layers [1]")
flags.DEFINE_integer("init_std", 0.05, "init_std")
flags.DEFINE_integer("init_scale", 1, "init_scale")
flags.DEFINE_bool('one_hot_embedding', True, 'using one_hot representation as rnn input')
flags.DEFINE_integer("embedding_size", 100, "word embedding size [50]")
flags.DEFINE_integer("hidden_size", 128, "rnn hidden size [128]")
flags.DEFINE_float("keep_prob", 1.0, "keep probability of drop out [0.9]")
flags.DEFINE_float("learning_rate", 0.002, "learning rate [0.001]")
flags.DEFINE_integer("batch_size", 512, "batch size to use during training [128]")
flags.DEFINE_float("clip_gradients", 5.0, "clip gradients to this norm [5.0]")
flags.DEFINE_integer("n_epochs", 10, "number of epoch to use during training [10]")
flags.DEFINE_boolean("epoch_save", True, "save checkpoint or not in each epoch [True]")
flags.DEFINE_integer("print_step", 100, "print step duraing training [100]")
flags.DEFINE_string("logs_dir", "logs/", "logs directory [logs/]")
flags.DEFINE_string("model_dir", "model/", "model directory [model/]")
flags.DEFINE_boolean("dir_clear", False, "clear the log and model directory")
flags.DEFINE_boolean("lr_annealing", False, "use lr annealing or not after each epoch [False]")
flags.DEFINE_string("current_task_name", 'url_self_prediction', "current task name [self_prediction]")
flags.DEFINE_integer("gpu_id", 0, "default gpu id [0]")
flags.DEFINE_integer("gpu_num", 1, "gpu_num")

FLAGS = flags.FLAGS

In [4]:
idxs = np.arange(0, len(show))
idxs = idxs[0:200000]
train_idxs, test_idxs = train_test_split(idxs, test_size=0.2, random_state=42)


In [5]:
res = {}
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
with tf.Session(graph=graph_to_use, config=config) as session:
    user2vec_model = User2VecModel(FLAGS, session, current_task_name='user2vec_model')
    user2vec_model.build_model()
    user2vec_model.build_model_summary()
    #user2vec_model.model_resotre()
    display(user2vec_model.model_summary())
    #user2vec_model.run([show,seq_len,isMovie,time], train_idxs, test_idxs)
    for var in tf.trainable_variables():
        res[var.name] = var.eval()
        


Initializing


,variable_name,variable_shape,parameters
0,global/Variable:0,[],1.0
1,gpu/embedding/embedding_layer/embedding_table:0,"[29000, 100]",2900000.0
2,gpu/rnn_lstm/rnn/multi_rnn_cell/cell_0/lstm_ce...,"[230, 512]",117760.0
3,gpu/rnn_lstm/rnn/multi_rnn_cell/cell_0/lstm_ce...,[512],512.0
4,gpu/rnn_lstm/rnn/multi_rnn_cell/cell_0/lstm_ce...,[128],128.0
5,gpu/rnn_lstm/rnn/multi_rnn_cell/cell_0/lstm_ce...,[128],128.0
6,gpu/rnn_lstm/rnn/multi_rnn_cell/cell_0/lstm_ce...,[128],128.0
7,gpu/prediction/conv_1d/he_uniform/W:0,"[1, 128, 29000]",3712000.0
8,gpu/prediction/conv_1d/B:0,[29000],29000.0


In [ ]:
#i, j, f, o = np.split(res['rnn/multi_rnn_cell/cell_0/lstm_cell/weights:0'], 4, 1)
#ib, jb, fb, ob = np.split(res['rnn/multi_rnn_cell/cell_0/lstm_cell/biases:0'], 4, 0)
#w = res['self_time_distributed_fc1/W:0']
#b = res['self_time_distributed_fc1/B:0']
